# Perceptron

Auteur: F. Rosenblatt

Reference: F. Rosenblatt 1958 *The Perceptron: a Probabilistic Model for Information Storage and Organization in the Brain* Psychological Review, 65, 386-408

Le modéle est constitué des éléments suivants:
- des *unités sensitives (S-units)*: réagissent à un stimuli extérieur (lumière, son, touché, ...)
    - retournent `0` ou `1`:
        - `1` si le signal d'entrée dépasse un seuil $\theta$
        - `0` sinon
- des *unités d'associations (A-units)*
    - retournent `0` ou `1`:
        - `1` si la somme des signaux d'entrée dépasse un seuil $\theta$
        - `0` sinon
- des *unités de réponse (R-units)*: sortie du réseau
    - retournent `1`, `-1` ou une valeur indéterminée:
        - `1` si la somme des signaux d'entrée est positive
        - `-1` si elle est négative
        - une valeur indéterminée si elle est égale à 0
- une *matrice d'intéractions*


Calcul de $e_j$ :

$$
e_j = \sum s_i w_{ij} = si1 . wij1 + si2 . wij2 + si3 . wij3 + si4 . wij4
$$

Calcul du nouveau poid $w$ :

$$
w_{ij}(t+1) = w_{ij}(t) + n (d_j - s_j) s_i
$$

Nombre de neurones associatifs (A-units) :
    
    Nombre d'unités sensitives : 4
    
    Entrez le pas d'apprentissage ƞ :
ƞ : 1

    Entrez les poids de depart des synapses du réseau :
wij0 : 
    
    Entrez le motif à apprendre :
si1 : si2 : si3 : si4 : 

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import matplotlib.patches as patches
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

import itertools

In [ ]:
# https://github.com/jeremiedecock/neural-network-figures.git
import nnfigs.core as fig

fig.draw_neural_network();

In [ ]:
# Init weights
initial_weights = np.array([0., 0., 0., 0., 2.])

# Set "learning rate" factor
nu = 1.

In [ ]:
def update_weights(w, x, yd):
    s = np.sum(x * w)
    yo = 1. if s >= 0. else 0.
    w = w + nu * (yd - yo) * x
    print(x, s, yo, yd, w)
    return w

def learn_examples(example_list, label_list, w, num_iterations):
    for it in range(num_iterations):
        print(it)
        for x, yd in zip(example_list, label_list):
            w = update_weights(w, np.array(x + (-1,)), yd)
    return w

In [ ]:
example_list = tuple(reversed(tuple(itertools.product((0., 1.), repeat=4))))
label_list = [1. if x == (1., 0., 0., 1.) else 0. for x in example_list]

print(example_list)
print(label_list)

In [ ]:
learn_examples(example_list, label_list, initial_weights, 4)